<a href="https://colab.research.google.com/github/yeialgov/MAYC_ML_Revenue_Prediction/blob/patch-1/ML_GACRP_PIPELINE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# download the libraries
import pandas as pd
from pandas import Series,DataFrame
import warnings
from copy import deepcopy
import os
warnings.filterwarnings('ignore')
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
from sklearn.preprocessing import LabelEncoder
import random
%matplotlib inline

# Getting the data

In [4]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [5]:
cd '/content/gdrive/My Drive/ReDi_DA_Project/preprocessed_ga_revenue_data'

/content/gdrive/My Drive/ReDi_DA_Project/preprocessed_ga_revenue_data


In [0]:
# load both of the datasets
traindata = pd.read_pickle('train_flat_no_hits.pkl')
testdata = pd.read_pickle('test_flat_no_hits.pkl')


In [0]:
# add a marker so that you know which dataset is which
traindata['dataset'] = 'train'
testdata['dataset'] = 'test'


In [0]:
traindata.drop('trafficSource.campaignCode',axis=1,inplace= True) #the test doesnt have trafficSource.campaignCode column, 
#so i droped it from the train set

In [0]:
# make sure the columns are in the same order in both train and test.
# the deepcopy is just to have a copy of the original even after joining them
col_order = list(set(traindata.columns.tolist() + testdata.columns.tolist()))
traindata = deepcopy(traindata[col_order])
testdata = deepcopy(testdata[col_order])


In [0]:
# append the two together
all_data = traindata[col_order].append(testdata)

In [10]:
all_data.head()

,totals.transactionRevenue,geoNetwork.subContinent,device.browserVersion,geoNetwork.region,totals.pageviews,fullVisitorId,trafficSource.isTrueDirect,device.mobileDeviceModel,device.language,trafficSource.adwordsClickInfo.page,trafficSource.adwordsClickInfo.gclId,device.mobileDeviceInfo,geoNetwork.continent,totals.sessionQualityDim,trafficSource.adwordsClickInfo.slot,totals.visits,visitStartTime,totals.transactions,device.flashVersion,geoNetwork.country,totals.hits,device.mobileInputSelector,device.operatingSystemVersion,trafficSource.medium,trafficSource.referralPath,trafficSource.adwordsClickInfo.isVideoAd,date,totals.timeOnSite,customDimension.value,socialEngagementType,trafficSource.adwordsClickInfo.criteriaParameters,geoNetwork.cityId,totals.totalTransactionRevenue,device.deviceCategory,totals.newVisits,customDimension.index,geoNetwork.latitude,channelGrouping,geoNetwork.metro,geoNetwork.city,device.operatingSystem,trafficSource.adContent,visitNumber,device.browser,dataset,trafficSource.keyword,device.mobileDeviceMarketingName,device.mobileDeviceBranding,device.screenColors,geoNetwork.longitude,geoNetwork.networkDomain,trafficSource.campaign,visitId,device.browserSize,device.screenResolution,totals.bounces,geoNetwork.networkLocation,trafficSource.source,trafficSource.adwordsClickInfo.adNetworkType,device.isMobile
0,NaN,Western Europe,not available in demo dataset,not available in demo dataset,1,3162355547410993243,NaN,not available in demo dataset,not available in demo dataset,NaN,NaN,not available in demo dataset,Europe,1,NaN,1,1508198450,NaN,not available in demo dataset,Germany,1,not available in demo dataset,not available in demo dataset,organic,NaN,NaN,20171016,NaN,EMEA,Not Socially Engaged,not available in demo dataset,not available in demo dataset,NaN,desktop,1,4,not available in demo dataset,Organic Search,not available in demo dataset,not available in demo dataset,Windows,NaN,1,Firefox,train,water bottle,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,(not set),(not set),1508198450,not available in demo dataset,not available in demo dataset,1,not available in demo dataset,google,NaN,False
1,NaN,Northern America,not available in demo dataset,California,2,8934116514970143966,NaN,not available in demo dataset,not available in demo dataset,NaN,NaN,not available in demo dataset,Americas,2,NaN,1,1508176307,NaN,not available in demo dataset,United States,2,not available in demo dataset,not available in demo dataset,referral,/a/google.com/transportation/mtv-services/bike...,NaN,20171016,28,North America,Not Socially Engaged,not available in demo dataset,not available in demo dataset,NaN,desktop,NaN,4,not available in demo dataset,Referral,San Francisco-Oakland-San Jose CA,Cupertino,Chrome OS,NaN,6,Chrome,train,NaN,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,(not set),(not set),1508176307,not available in demo dataset,not available in demo dataset,NaN,not available in demo dataset,sites.google.com,NaN,False
2,NaN,Northern America,not available in demo dataset,not available in demo dataset,2,7992466427990357681,True,not available in demo dataset,not available in demo dataset,NaN,NaN,not available in demo dataset,Americas,1,NaN,1,1508201613,NaN,not available in demo dataset,United States,2,not available in demo dataset,not available in demo dataset,(none),NaN,NaN,20171016,38,North America,Not Socially Engaged,not available in demo dataset,not available in demo dataset,NaN,mobile,1,4,not available in demo dataset,Direct,not available in demo dataset,not available in demo dataset,Android,NaN,1,Chrome,train,NaN,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,windjammercable.net,(not set),1508201613,not available in demo dataset,not available in demo dataset,NaN,not available in demo dataset,(direct),NaN,True
3,N

In [0]:
all_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2109926 entries, 0 to 401588
Data columns (total 60 columns):
 #   Column                                             Dtype 
---  ------                                             ----- 
 0   device.browser                                     object
 1   totals.transactionRevenue                          object
 2   trafficSource.adwordsClickInfo.gclId               object
 3   totals.hits                                        object
 4   totals.timeOnSite                                  object
 5   trafficSource.adwordsClickInfo.criteriaParameters  object
 6   trafficSource.medium                               object
 7   trafficSource.adwordsClickInfo.page                object
 8   device.language                                    object
 9   geoNetwork.metro                                   object
 10  totals.sessionQualityDim                           object
 11  device.mobileInputSelector                         object
 12  t

In [0]:
def values_missing(check):
    total = check.isnull().sum().sort_values(ascending = False) # getting the sum of null values and ordering
    percent = (check.isnull().sum() / check.isnull().count() * 100 ).sort_values(ascending = False) #getting the percent and order of null
    df_train = pd.concat([total, percent], axis=1, keys=['Total', 'Percent']) # Concatenating the total and percent
    print("Total and percent of missing values: ")
    print (df_train[~(df_train['Total'] == 0)]) # Returning values of nulls different of 0
    
    return

In [0]:
values_missing(all_data)

Total and percent of missing values: 
                                                Total    Percent
totals.totalTransactionRevenue                2086818  98.904796
totals.transactionRevenue                     2086818  98.904796
totals.transactions                           2085062  98.821570
trafficSource.adwordsClickInfo.slot           2024047  95.929762
trafficSource.adwordsClickInfo.isVideoAd      2024047  95.929762
trafficSource.adwordsClickInfo.adNetworkType  2024047  95.929762
trafficSource.adwordsClickInfo.page           2024047  95.929762
trafficSource.adwordsClickInfo.gclId          2023891  95.922369
trafficSource.adContent                       1643600  77.898467
trafficSource.isTrueDirect                    1426999  67.632656
trafficSource.referralPath                    1142073  54.128581
trafficSource.keyword                         1093006  51.803049
totals.timeOnSite                             1057980  50.142991
totals.bounces                                105567

In [0]:
def see_data(data, data_type=object, limit=5): #seting the function with data,show 5 uniques if available 
    n = data.select_dtypes(include=data_type) #selecting the desired data type
    for column in n.columns: #initializing the loop
        print("______________________________________________________________")
        print("Name of column ", column, ': \n', "Uniques: ", data[column].unique()[:limit], "\n",
              " | >> Total nulls: ", (round(data[column].isnull().sum() / len(data[column]) * 100,1)),
              " | >> Total unique values: ", all_data.nunique()[column]) #print the data and % of nulls)
        print("_________________________________________________________")

In [0]:
#see_data(traindata)

______________________________________________________________
Name of column  device.browser : 
 Uniques:  ['Firefox' 'Chrome' 'Safari' 'UC Browser' 'Internet Explorer'] 
  | >> Total nulls:  0.0  | >> Total unique values:  161
_________________________________________________________
______________________________________________________________
Name of column  totals.transactionRevenue : 
 Uniques:  [nan '15190000' '8000000' '57300000' '18230000'] 
  | >> Total nulls:  98.9  | >> Total unique values:  8219
_________________________________________________________
______________________________________________________________
Name of column  trafficSource.adwordsClickInfo.gclId : 
 Uniques:  [nan
 'Cj0KCQjwsZHPBRClARIsAC-VMPBHdNF2oMOgh6Xp6YhjXWSk11sDu3eCo4De_u6l4xafqgrPVPSGYdkaArkiEALw_wcB'
 'CODVoMjJ9tYCFUIvgQod_dsKEA'
 'Cj0KCQjwsZHPBRClARIsAC-VMPA4CVJtDhu1lYkB0AR1hje1goUMHQZNJMrsSWD57pLnv1KE_L7pCEcaAunKEALw_wcB'
 'Cj0KCQjwsZHPBRClARIsAC-VMPDlLD6kS4tmqFGZjMUqyerwe0WDaULaAKNkj-gwuvPn

In [0]:
drop_col_list = ['socialEngagementType', 'device.browserVersion', 'device.browserSize', 'device.operatingSystemVersion',
                 'device.mobileDeviceBranding', 'device.mobileDeviceModel', 'device.mobileInputSelector', 
                 'device.mobileDeviceInfo', 'device.mobileDeviceMarketingName', 'device.flashVersion', 'device.language', 
                 'device.screenColors', 'device.screenResolution', 'geoNetwork.cityId', 'geoNetwork.latitude', 
                 'geoNetwork.longitude', 'geoNetwork.networkLocation', 'trafficSource.campaign', 'trafficSource.referralPath', 
                 'trafficSource.adwordsClickInfo.page','trafficSource.adwordsClickInfo.slot', 
                 'trafficSource.adwordsClickInfo.criteriaParameters', 'trafficSource.adContent', 'trafficSource.adwordsClickInfo.gclId', 
                 'customDimension.index','totals.visits','trafficSource.keyword']

In [0]:
all_data.drop(drop_col_list, axis=1, inplace=True)  #to drop them

In [0]:
all_data['totals.pageviews'].fillna(1, inplace=True) #filling NA's with 1(since least for a visit to the site means 1 page seen)
all_data['totals.newVisits'].fillna(0, inplace=True) #filling NA's with 0(0 is suitable for Nan here)
all_data['totals.bounces'].fillna(0, inplace=True)   #filling NA's with 0(0 is suitable for Nan here)
all_data["totals.transactionRevenue"] = all_data["totals.transactionRevenue"].fillna(0.0).astype(float) #filling NA with zero
all_data["totals.totalTransactionRevenue"] = all_data["totals.totalTransactionRevenue"].fillna(0.0).astype(float) #filling NA with zero
all_data["totals.transactionRevenue"] = all_data["totals.transactionRevenue"].fillna(0.0).astype(float) #filling NA with zero
all_data["totals.transactions"] = all_data["totals.transactions"].fillna(0).astype(int) #filling NA with zero
all_data['trafficSource.isTrueDirect'].fillna(False, inplace=True) # filling boolean with False is its NAn
all_data['trafficSource.adwordsClickInfo.isVideoAd'].fillna(True, inplace=True) # filling boolean is its NAn
all_data['customDimension.value'].fillna('(not set)', inplace=True) # filling NA with '(not set)'
all_data['trafficSource.adwordsClickInfo.adNetworkType'].fillna('(not set)', inplace=True) # filling NA with '(not set)'
all_data['totals.timeOnSite'].fillna(0, inplace=True) #filling NA's with 0(0 is suitable for Nan here)
all_data['totals.sessionQualityDim'].fillna(0, inplace=True) #filling NA's with 0(0 is suitable for Nan here)
#To convert
all_data['totals.pageviews'] = all_data['totals.pageviews'].astype(int) # setting numerical column as integer
all_data['totals.newVisits'] = all_data['totals.newVisits'].astype(int) # setting numerical column as integer
all_data['totals.bounces'] = all_data['totals.bounces'].astype(int)  # setting numerical column as integer
all_data["totals.hits"] = all_data["totals.hits"].astype(float) # setting numerical to float
all_data['totals.sessionQualityDim'] = all_data['totals.sessionQualityDim'].astype(int)
all_data['totals.timeOnSite'] = all_data['totals.timeOnSite'].astype(int)
all_data['date'] = pd.to_datetime(all_data["date"], format="%Y%m%d") #convert to datetime

In [0]:
#all_data['geoNetwork.metro'].replace({"not available in demo dataset": "(not set)"}, inplace=True)

In [14]:
all_data.isnull().sum().sum()

0

In [15]:
all_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2109926 entries, 0 to 401588
Data columns (total 33 columns):
 #   Column                                        Dtype         
---  ------                                        -----         
 0   trafficSource.source                          object        
 1   customDimension.value                         object        
 2   geoNetwork.city                               object        
 3   totals.transactionRevenue                     float64       
 4   geoNetwork.country                            object        
 5   totals.newVisits                              int64         
 6   totals.pageviews                              int64         
 7   visitNumber                                   int64         
 8   trafficSource.isTrueDirect                    bool          
 9   dataset                                       object        
 10  geoNetwork.continent                          object        
 11  channelGrouping          

In [0]:
'''
columns to hotcode

channelGrouping                               object  unique values:  8
date                                          int64  - convert to date time
fullVisitorId                                 object - organisers insist to be left as str
device.browser                                object - unique values:  129
device.operatingSystem                        object - unique values:  24
device.isMobile                               bool   - unique values:  2
device.deviceCategory                         object - unique values:  3
geoNetwork.continent                          object - unique values:  6
geoNetwork.subContinent                       object - unique values:  23
geoNetwork.country                            object - unique values:  228
geoNetwork.region                             object - unique values:  284
geoNetwork.metro                              object - unique values:  123
geoNetwork.city                               object - unique values:  956
geoNetwork.networkDomain                      object - unique values:  41982
totals.sessionQualityDim                      object - convert to interger
totals.timeOnSite                             object - convert to interger
trafficSource.source                          object - unique values:  345
trafficSource.medium                          object - unique values:  7
trafficSource.isTrueDirect                    bool   - unique values:  2 
trafficSource.adwordsClickInfo.adNetworkType  object - unique values:  3
trafficSource.adwordsClickInfo.isVideoAd      bool   - unique values:  2 
customDimension.value                         object - unique values:  5
'''

# Model to search the visitors who made transactions

In [0]:
def is_transaction(n):
  if n == 0    :    return 0
  else         :    return 1

all_data['is_transaction']=all_data['totals.transactions'].apply(is_transaction)

In [28]:
all_data['is_transaction'].value_counts()


0    2085062
1      24864
Name: is_transaction, dtype: int64

In [0]:
TARGET = 'is_transaction'
CAT_COLS = all_data.columns[all_data.dtypes == object].tolist()
CONT_COLS = [c for c in all_data.columns if c not in CAT_COLS and c != TARGET]


In [36]:
CAT_COLS

['trafficSource.source',
 'customDimension.value',
 'geoNetwork.city',
 'geoNetwork.country',
 'dataset',
 'geoNetwork.continent',
 'channelGrouping',
 'trafficSource.adwordsClickInfo.adNetworkType',
 'device.operatingSystem',
 'device.browser',
 'fullVisitorId',
 'geoNetwork.metro',
 'geoNetwork.subContinent',
 'geoNetwork.region',
 'device.deviceCategory',
 'geoNetwork.networkDomain',
 'trafficSource.medium']

In [0]:

CONT_COLS.remove('totals.transactions')

In [22]:
all_data = deepcopy(all_data[CAT_COLS + CONT_COLS + [TARGET]])

all_data.shape

(2109926, 33)

In [0]:
#cat_cols = ['device.browser', 'totals.hits','trafficSource.medium', 'geoNetwork.metro', 'customDimension.value',
 #           'visitId','trafficSource.isTrueDirect','geoNetwork.region','device.deviceCategory', 'geoNetwork.city', 
  #          'channelGrouping','trafficSource.adwordsClickInfo.adNetworkType', 'fullVisitorId',
   #         'device.isMobile', 'geoNetwork.networkDomain','geoNetwork.subContinent', 'trafficSource.adwordsClickInfo.isVideoAd',
    #        'device.operatingSystem', 'geoNetwork.continent','geoNetwork.country','trafficSource.source']
   

In [0]:
leb = LabelEncoder()
for col in CAT_COLS:
    all_data[col] = leb.fit_transform(all_data[col])

In [25]:
all_data['dataset'].value_counts()

1    1708337
0     401589
Name: dataset, dtype: int64

In [33]:
train = all_data[all_data['dataset']==1]
test = all_data[all_data['dataset']==0]
train.drop('dataset',axis=1,inplace=True)
test.drop('dataset',axis=1, inplace=True)

(401589, 32)

In [0]:

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.metrics import accuracy_score as accuracy
from sklearn.metrics import r2_score as r2
from sklearn.metrics import mean_squared_error as mse
from sklearn.metrics import mean_absolute_error as mae
rmse = lambda true, pred: mse(true, pred) ** 0.5

In [0]:
#CAT_COLS.remove('dataset')
CONT_COLS.remove('date')

In [0]:
x_train = train[CAT_COLS + CONT_COLS]
x_test = test[CAT_COLS + CONT_COLS]
y_train = train[[TARGET]]
y_test = test[[TARGET]]

In [48]:
x_train.shape, x_test.shape, y_train.shape, y_test.shape

((1708337, 30), (401589, 30), (1708337, 1), (401589, 1))

In [0]:
model = RandomForestClassifier( 
    n_estimators=50, # Hyperparameter 1
    max_depth=2,      # Hyperparameter 2
    random_state=0
)

In [0]:
trained_model = model.fit(x_train, y_train)

In [0]:
def get_error(pred, true):
    print(f'RMSE:       {rmse(true, pred)}')
    print(f'MAE:        {mae(true, pred)}')
    print(f'r2:         {r2(true, pred)}')
    print(f'accuracy:   {accuracy(true, pred)}')

In [61]:
get_error(pred = trained_model.predict(x_test),
          true = y_test)

RMSE:       0.06527307918539964
MAE:        0.004260574866343451
r2:         0.724299539549079
accuracy:   0.9957394251336565
